In [1]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt
import seaborn           as sns

In [2]:
from os import path

data_dir  = 'data/spaceship-titanic'
data_file = 'train.csv'
data_path = path.join(data_dir, data_file)

df = pd.read_csv(data_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [3]:
labels = df.pop('Transported')
df.drop(columns=['PassengerId', 'Name'], inplace=True)

# NÃO POSSO DROPAR `PassengerId`, TEM INFORMAÇÃO IMPORTANTE AÍ

Vide https://www.kaggle.com/competitions/spaceship-titanic/data

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Cabin         8494 non-null   object 
 3   Destination   8511 non-null   object 
 4   Age           8514 non-null   float64
 5   VIP           8490 non-null   object 
 6   RoomService   8512 non-null   float64
 7   FoodCourt     8510 non-null   float64
 8   ShoppingMall  8485 non-null   float64
 9   Spa           8510 non-null   float64
 10  VRDeck        8505 non-null   float64
dtypes: float64(6), object(5)
memory usage: 747.2+ KB


In [5]:
#      HomePlanet    6606 non-null   object 
#  2   CryoSleep     6606 non-null   object 
#  3   Cabin         6606 non-null   object 
#  4   Destination   6606 non-null   object 
#  6   VIP           6606 non-null   object
print(df['HomePlanet'].unique())
print(df['CryoSleep'].unique())
print(df['Cabin'].unique().shape)
print(df['Destination'].unique())
print(df['VIP'].unique())

['Europa' 'Earth' 'Mars' nan]
[False True nan]
(6561,)
['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]
[False True nan]


## O QUE SERIA NECESSÁRIO FAZER:

### - Repensar o dropna() - muitas entradas podem ser mantidas (por exemplo, possuir Name nulo não importa)
### - One-hot dummy encoding em HomePlanet e Destination
### - Binary encoding em CryoSleep
### - Separar Cabin em três atributos diferentes e tratá-los de acordo 

## O QUE DÁ PRA FAZER AGORA:

### - Testar um classificador supervisionado e testar um classificador não-supervisionado
### - Testar o classificador não-supervisionado contra a ground-truth

### Apenas droppando os valores nulos e boa

In [6]:
df.dropna(inplace=True)
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6764 entries, 0 to 6763
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    6764 non-null   object 
 1   CryoSleep     6764 non-null   object 
 2   Cabin         6764 non-null   object 
 3   Destination   6764 non-null   object 
 4   Age           6764 non-null   float64
 5   VIP           6764 non-null   object 
 6   RoomService   6764 non-null   float64
 7   FoodCourt     6764 non-null   float64
 8   ShoppingMall  6764 non-null   float64
 9   Spa           6764 non-null   float64
 10  VRDeck        6764 non-null   float64
dtypes: float64(6), object(5)
memory usage: 581.4+ KB


In [8]:
print(df['HomePlanet'].unique())
print(df['CryoSleep'].unique())
print(df['Cabin'].unique().shape)
print(df['Destination'].unique())
print(df['VIP'].unique())

['Europa' 'Earth' 'Mars']
[False True]
(5413,)
['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e']
[False True]


## AQUI PRA CIMA EU PODIA VISUALIZAR O CONJUNTO

### Convertendo atributos com dummy encoding

In [9]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Convertendo HomePlanet
HomePlanet     = np.reshape(df['HomePlanet'].values, (-1, 1))
HomePlanet_new = OneHotEncoder(drop='first').fit_transform(HomePlanet)
HomePlanet_new = pd.DataFrame.sparse.from_spmatrix(HomePlanet_new, columns=['FromEuropa', 'FromMars'])
HomePlanet_new = HomePlanet_new.sparse.to_dense()

# Convertendo Destination
Destination = np.reshape(df['Destination'].values, (-1, 1))
Destination_new = OneHotEncoder(drop='first').fit_transform(Destination)
Destination_new = pd.DataFrame.sparse.from_spmatrix(Destination_new, columns=['ToPSO', 'ToTrappist'])
Destination_new = Destination_new.sparse.to_dense()

In [10]:
df = pd.concat([df, HomePlanet_new, Destination_new], axis=1, join='inner')
df.drop(columns=['HomePlanet', 'Destination'], inplace=True)

In [11]:
df

,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,FromEuropa,FromMars,ToPSO,ToTrappist
0,False,B/0/P,39.0,False,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,False,F/0/S,24.0,False,109.0,9.0,25.0,549.0,44.0,0.0,0.0,0.0,1.0
2,False,A/0/S,58.0,True,43.0,3576.0,0.0,6715.0,49.0,1.0,0.0,0.0,1.0
3,False,A/0/S,33.0,False,0.0,1283.0,371.0,3329.0,193.0,1.0,0.0,0.0,1.0
4,False,F/1/S,16.0,False,303.0,70.0,151.0,565.0,2.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6759,False,A/98/P,41.0,True,0.0,6819.0,0.0,1643.0,74.0,1.0,0.0,0.0,0.0
6760,True,G/1499/S,18.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6761,False,G/1500/S,26.0,False,0.0,0.0,1872.0,1.0,0.0,0.0,0.0,0.0,1.0
6762,False,E/608/S,32.0,False,0.0,1049.0,0.0,353.0,3235.0,1.0,0.0,0.0,0.0


### Convertendo atributos binários em numéricos

In [12]:
binary_attribs = ['CryoSleep', 'VIP']

In [13]:
for attrib in binary_attribs:
    df = pd.concat([
        df.drop(columns=attrib),
        df[attrib].apply(lambda x: float(x))
    ], axis=1)

In [14]:
df

,Cabin,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,FromEuropa,FromMars,ToPSO,ToTrappist,CryoSleep,VIP
0,B/0/P,39.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,F/0/S,24.0,109.0,9.0,25.0,549.0,44.0,0.0,0.0,0.0,1.0,0.0,0.0
2,A/0/S,58.0,43.0,3576.0,0.0,6715.0,49.0,1.0,0.0,0.0,1.0,0.0,1.0
3,A/0/S,33.0,0.0,1283.0,371.0,3329.0,193.0,1.0,0.0,0.0,1.0,0.0,0.0
4,F/1/S,16.0,303.0,70.0,151.0,565.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6759,A/98/P,41.0,0.0,6819.0,0.0,1643.0,74.0,1.0,0.0,0.0,0.0,0.0,1.0
6760,G/1499/S,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
6761,G/1500/S,26.0,0.0,0.0,1872.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6762,E/608/S,32.0,0.0,1049.0,0.0,353.0,3235.0,1.0,0.0,0.0,0.0,0.0,0.0


### Lidando com `Cabin`

In [15]:
# The cabin number where the passenger is staying.
# Takes the form deck/num/side, where side can be either P for Port or S for Starboard.

In [16]:
df[['CabinDeck', 'CabinNumber', 'CabinSide']] = df['Cabin'].str.split('/', expand=True)
df.drop(columns='Cabin', inplace=True)

In [19]:
np.unique(df['CabinDeck'].values, return_counts=True)

(array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'], dtype=object),
 array([ 213,  641,  599,  381,  700, 2208, 2019,    3]))

In [21]:
df[ df['CabinDeck'] == 'T' ]

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,FromEuropa,FromMars,ToPSO,ToTrappist,CryoSleep,VIP,CabinDeck,CabinNumber,CabinSide
1729,42.0,0.0,1829.0,2.0,3133.0,2447.0,1.0,0.0,0.0,1.0,0.0,0.0,T,1,P
2095,33.0,0.0,28.0,0.0,6841.0,543.0,1.0,0.0,0.0,1.0,0.0,0.0,T,2,P
2116,38.0,0.0,3135.0,0.0,26.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,T,3,P


## Têm mais coisa pra fazer com `Cabin`, mas se eu converter tudo agora eu me ferro depois pra visualizar os dados